In [15]:
from io import BytesIO

import requests
import pandas as pd

from decimal import Decimal

from yandex_geocoder import Client

import folium

In [16]:
def prepro_url(url):
    csv_export_url = url.replace('/edit#gid=', '/export?format=csv&gid=')
    return csv_export_url
    

In [17]:
# sheet_url = 'https://docs.google.com/spreadsheets/d/1iSrqedH09Ck2bTiTIo57whu94daXHMls6ig-748UC5s/edit#gid=0'
# csv_export_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

pers_url = 'https://docs.google.com/spreadsheets/d/1iSrqedH09Ck2bTiTIo57whu94daXHMls6ig-748UC5s/edit#gid=0'


r = requests.get(prepro_url(pers_url))
data = r.content

In [18]:
df_pers = pd.read_csv(BytesIO(data))

In [19]:
# df_pers

In [20]:
letter_url = 'https://docs.google.com/spreadsheets/d/1iSrqedH09Ck2bTiTIo57whu94daXHMls6ig-748UC5s/edit#gid=716074019'
r = requests.get(prepro_url(letter_url))
data = r.content

In [21]:
df_letter = pd.read_csv(BytesIO(data))

In [22]:
df_letter

,address
0,г. Москва
1,г. Химки
2,г. Берлин


In [23]:
# geolocator = Nominatim(user_agent="freedom-sach-map")

In [24]:
api_key = '4daecbaf-37cc-47f6-9602-55e281126741'
client = Client(api_key)

In [25]:
lat = []
long = []

for i in df_pers['address'].to_list():
    coordinates = client.coordinates(i)
    long.append(str(coordinates[0]))
    lat.append(str(coordinates[1]))

In [26]:
df_pers['lat'] = lat
df_pers['long'] = long

In [27]:
lat = []
long = []

for i in df_letter['address'].to_list():
    coordinates = client.coordinates(i)
    long.append(str(coordinates[0]))
    lat.append(str(coordinates[1]))
    

df_letter['lat'] = lat
df_letter['long'] = long

In [31]:
moscow = [55.60, 37.618]

m1 = folium.Map(location=moscow,
                tiles='openstreetmap',
                # tiles='	http://c.tile.stamen.com/watercolor/{z}/{x}/{y}.jpg',
                # attr='Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under CC BY SA',
                zoom_start=9)

In [32]:
m1

In [33]:
for i in sorted(df_pers['name'].unique()):
    temp = df_pers[df_pers['name']==i]
    
    lat = temp['lat'].astype(float).to_list()[0]
    long = temp['long'].astype(float).to_list()[0]
    
    folium.Marker(
        location=[lat, long],
        # popup='<i>Здесь был Вася</i>',
        tooltip=f'{i}').add_to(m1)

In [34]:
m1

In [35]:
for i in sorted(df_letter['address'].unique()):
    temp = df_letter[df_letter['address']==i]
    
    lat = temp['lat'].astype(float).to_list()[0]
    long = temp['long'].astype(float).to_list()[0]
    
    folium.Marker(
        location=[lat, long],
        # popup='<i>Здесь был Вася</i>',
        icon=folium.Icon(color="red"),
        tooltip=f'{i}').add_to(m1)

In [36]:
m1

In [38]:
m1.save('docs/index.html')